In [30]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Activation, Input, GlobalAveragePooling2D, GaussianNoise
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
import pandas as pd
import os

In [31]:
train_dir = 'data_nocc/train'
test_dir = 'data_nocc/test'

In [32]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   validation_split=0.2,
                                   featurewise_center=False,
                                   featurewise_std_normalization=False,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True)

In [33]:
valid_datagen = ImageDataGenerator(rescale=1. / 255,
                                   validation_split=0.2)

In [34]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [35]:
train_dataset = train_datagen.flow_from_directory(directory=train_dir,
                                                  target_size=(256, 256),
                                                  class_mode='categorical',
                                                  subset='training',
                                                  batch_size=64)

Found 3040 images belonging to 12 classes.


In [36]:
valid_dataset = valid_datagen.flow_from_directory(directory=train_dir,
                                                  target_size=(256, 256),
                                                  class_mode='categorical',
                                                  subset='validation',
                                                  batch_size=64)

Found 756 images belonging to 12 classes.


In [37]:
test_dataset = test_datagen.flow_from_directory(directory=test_dir,
                                                  target_size=(256, 256),
                                                  class_mode='categorical',
                                                  batch_size=64)

Found 1647 images belonging to 12 classes.


In [38]:
image_shape = (256, 256, 3)
num_classes = 12
epochs = 60
lr = 0.0001

Model w/o ImageNet Pretraining

In [39]:
base_model = ResNet50(input_shape=image_shape, include_top=False, weights=None)

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(BatchNormalization(momentum=0.98))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization(momentum=0.98))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes,activation='softmax'))

In [40]:
os.makedirs('weights_nocc', exist_ok=True)
mcp = ModelCheckpoint('weights_nocc/resnet50_model.h5',
                             save_best_only=True,
                             verbose=1,
                             mode='min',
                             moniter='val_loss')

lrd = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=6, 
                              verbose=1, 
                              min_delta=0.0001)
es = EarlyStopping(monitor='val_loss', 
                          min_delta=0, 
                          patience=3, 
                          verbose=1, 
                          restore_best_weights=True)

In [41]:
sgd = SGD(lr=0.01,momentum=0.9, decay=0.0001, nesterov=True)
adam = Adam(lr=lr, decay=1e-6)

In [42]:
steps_per_epoch = train_dataset.n // train_dataset.batch_size
validation_steps = train_dataset.n // train_dataset.batch_size

In [43]:
model.compile(optimizer=adam, 
              loss='categorical_crossentropy', 
               metrics=['accuracy'])
history = model.fit(x=train_dataset,
                 validation_data=valid_dataset,
                 epochs=epochs,
                 callbacks=[lrd, mcp],
                 steps_per_epoch=steps_per_epoch,
                 validation_steps=validation_steps)

ResourceExhaustedError: OOM when allocating tensor with shape[131072,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node dense_6/kernel/Initializer/random_uniform/RandomUniform (defined at /home/software/anaconda3/2019.10_powerai/envs/powerai/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Original stack trace for 'dense_6/kernel/Initializer/random_uniform/RandomUniform':
  File "/home/software/anaconda3/2019.10_powerai/envs/powerai/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/software/anaconda3/2019.10_powerai/envs/powerai/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/software/anaconda3/2019.10_powerai/envs/powerai/lib/python3.7/site-packages/ipykernel_launcher.p

In [ ]:
print("Evaluate on test data")
results = model.evaluate(test_dataset)
print("test loss, test acc:", results)

In [ ]:
os.makedirs('history_nocc', exist_ok=True)
file_name = 'history_nocc/resnet50_hist.csv'
with open(file_name, mode='w') as f:
    pd.DataFrame(history.history).to_csv(f)